<a href="https://colab.research.google.com/github/Mubashar228/Hotel-Booking-Cancellation/blob/main/Hotel_Booking_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
cpath=("/content/hotel_booking.csv")

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("Hotel_Booking_Canceletion").getOrCreate()

In [ ]:
spark

In [ ]:
df=spark.read.csv(cpath,header=True,inferSchema=True)

In [ ]:
df.show()

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+------+---------------------------+-------------------------+------------------+-----------------------+--------------------+--------------------+------------+----------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|

In [ ]:
df.show(3)

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+----+---------------------------+-------------------------+------------------+-----------------------+--------------+--------------------+------------+----------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_

In [ ]:
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: double (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

In [ ]:
df

DataFrame[hotel: string, is_canceled: int, lead_time: int, arrival_date_year: int, arrival_date_month: string, arrival_date_week_number: int, arrival_date_day_of_month: int, stays_in_weekend_nights: int, stays_in_week_nights: int, adults: int, children: double, babies: int, meal: string, country: string, market_segment: string, distribution_channel: string, is_repeated_guest: int, previous_cancellations: int, previous_bookings_not_canceled: int, reserved_room_type: string, assigned_room_type: string, booking_changes: int, deposit_type: string, agent: double, company: double, days_in_waiting_list: int, customer_type: string, adr: double, required_car_parking_spaces: int, total_of_special_requests: int, reservation_status: string, reservation_status_date: date, name: string, email: string, phone-number: string, credit_card: string]

In [ ]:
# Data Cleaning and Feature Engineering

In [ ]:
from pyspark.sql.functions import col

In [ ]:
df.na.drop()

DataFrame[hotel: string, is_canceled: int, lead_time: int, arrival_date_year: int, arrival_date_month: string, arrival_date_week_number: int, arrival_date_day_of_month: int, stays_in_weekend_nights: int, stays_in_week_nights: int, adults: int, children: double, babies: int, meal: string, country: string, market_segment: string, distribution_channel: string, is_repeated_guest: int, previous_cancellations: int, previous_bookings_not_canceled: int, reserved_room_type: string, assigned_room_type: string, booking_changes: int, deposit_type: string, agent: double, company: double, days_in_waiting_list: int, customer_type: string, adr: double, required_car_parking_spaces: int, total_of_special_requests: int, reservation_status: string, reservation_status_date: date, name: string, email: string, phone-number: string, credit_card: string]

In [ ]:
selected_col=[
    "lead_time","hotel","previous_cancellations","previous_bookings_not_canceled", "booking_changes","deposit_type","customer_type","market_segment","required_car_parking_spaces","total_of_special_requests","is_canceled"
]

In [ ]:
df1=df.select(selected_col)
df1.show(3)

+---------+------------+----------------------+------------------------------+---------------+------------+-------------+--------------+---------------------------+-------------------------+-----------+
|lead_time|       hotel|previous_cancellations|previous_bookings_not_canceled|booking_changes|deposit_type|customer_type|market_segment|required_car_parking_spaces|total_of_special_requests|is_canceled|
+---------+------------+----------------------+------------------------------+---------------+------------+-------------+--------------+---------------------------+-------------------------+-----------+
|      342|Resort Hotel|                     0|                             0|              3|  No Deposit|    Transient|        Direct|                          0|                        0|          0|
|      737|Resort Hotel|                     0|                             0|              4|  No Deposit|    Transient|        Direct|                          0|                        

In [ ]:
df.show(2)

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+-------------+--------------------+------------+----------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_wa

In [ ]:
df.select("hotel").distinct().show()

+------------+
|       hotel|
+------------+
|  City Hotel|
|Resort Hotel|
+------------+



In [ ]:
df1.describe().show()

+-------+------------------+------------+----------------------+------------------------------+-------------------+------------+---------------+--------------+---------------------------+-------------------------+-------------------+
|summary|         lead_time|       hotel|previous_cancellations|previous_bookings_not_canceled|    booking_changes|deposit_type|  customer_type|market_segment|required_car_parking_spaces|total_of_special_requests|        is_canceled|
+-------+------------------+------------+----------------------+------------------------------+-------------------+------------+---------------+--------------+---------------------------+-------------------------+-------------------+
|  count|            119390|      119390|                119390|                        119390|             119390|      119390|         119390|        119390|                     119390|                   119390|             119390|
|   mean|104.01141636652986|        NULL|   0.08711784906608594|

In [ ]:
df.select("lead_time").describe().show()

+-------+------------------+
|summary|         lead_time|
+-------+------------------+
|  count|            119390|
|   mean|104.01141636652986|
| stddev|106.86309704798786|
|    min|                 0|
|    max|               737|
+-------+------------------+



In [ ]:
df1.select("lead_time").show()

+---------+
|lead_time|
+---------+
|      342|
|      737|
|        7|
|       13|
|       14|
|       14|
|        0|
|        9|
|       85|
|       75|
|       23|
|       35|
|       68|
|       18|
|       37|
|       68|
|       37|
|       12|
|        0|
|        7|
+---------+
only showing top 20 rows



In [ ]:
df.select("deposit_type").distinct().show()

+------------+
|deposit_type|
+------------+
|  Non Refund|
|  Refundable|
|  No Deposit|
+------------+



In [ ]:
from pyspark.sql.functions import col,sum
df1.select([sum(col(c).isNull().cast("int")).alias(c) for c in df1.columns]).show()

+---------+-----+----------------------+------------------------------+---------------+------------+-------------+--------------+---------------------------+-------------------------+-----------+
|lead_time|hotel|previous_cancellations|previous_bookings_not_canceled|booking_changes|deposit_type|customer_type|market_segment|required_car_parking_spaces|total_of_special_requests|is_canceled|
+---------+-----+----------------------+------------------------------+---------------+------------+-------------+--------------+---------------------------+-------------------------+-----------+
|        0|    0|                     0|                             0|              0|           0|            0|             0|                          0|                        0|          0|
+---------+-----+----------------------+------------------------------+---------------+------------+-------------+--------------+---------------------------+-------------------------+-----------+



***Feature  Engineering(Categorical columns ko Encode karna)***

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
from pyspark.ml.feature import StringIndexer

# Define the StringIndexer
Indexer = StringIndexer(inputCols=["hotel","deposit_type","customer_type","market_segment"],
                        outputCols=["hotel_index","deposit_type_index","customer_type_index","market_segment_index"])

# Fit and transform using the 'Indexer' variable (uppercase 'I') you defined
data = Indexer.fit(df1).transform(df1)

# Now select the columns, including the newly created indexed columns
data.select("hotel","deposit_type","customer_type","market_segment","hotel_index","deposit_type_index","customer_type_index","market_segment_index").distinct().show(5)

+------------+------------+---------------+--------------+-----------+------------------+-------------------+--------------------+
|       hotel|deposit_type|  customer_type|market_segment|hotel_index|deposit_type_index|customer_type_index|market_segment_index|
+------------+------------+---------------+--------------+-----------+------------------+-------------------+--------------------+
|Resort Hotel|  No Deposit|          Group|        Direct|        1.0|               0.0|                3.0|                 3.0|
|  City Hotel|  No Deposit|       Contract|     Online TA|        0.0|               0.0|                2.0|                 0.0|
|Resort Hotel|  Refundable|          Group|        Groups|        1.0|               2.0|                3.0|                 2.0|
|  City Hotel|  No Deposit|          Group|        Direct|        0.0|               0.0|                3.0|                 3.0|
|Resort Hotel|  Non Refund|Transient-Party|        Direct|        1.0|             

In [ ]:
indexers = [
    StringIndexer(inputCol="hotel", outputCol="hotel_index"),
    StringIndexer(inputCol="deposit_type", outputCol="deposit_type_index"),
    StringIndexer(inputCol="customer_type", outputCol="customer_type_index"),
    StringIndexer(inputCol="market_segment", outputCol="market_segment_index")
]

# Apply transformations
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=indexers)
df1 = pipeline.fit(df1).transform(df1)

In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_columns = ["lead_time", "hotel_index", "previous_cancellations",
                   "previous_bookings_not_canceled", "booking_changes",
                   "deposit_type_index", "customer_type_index",
                   "market_segment_index", "required_car_parking_spaces",
                   "total_of_special_requests"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_xy = assembler.transform(df1).select("features", "is_canceled")

df_xy.show(5)

+--------------------+-----------+
|            features|is_canceled|
+--------------------+-----------+
|(10,[0,1,4,7],[34...|          0|
|(10,[0,1,4,7],[73...|          0|
|(10,[0,1,7],[7.0,...|          0|
|(10,[0,1,7],[13.0...|          0|
|(10,[0,1,9],[14.0...|          0|
+--------------------+-----------+
only showing top 5 rows



In [ ]:
df_xy.select("features", "is_canceled").show(truncate=False)

+----------------------------------+-----------+
|features                          |is_canceled|
+----------------------------------+-----------+
|(10,[0,1,4,7],[342.0,1.0,3.0,3.0])|0          |
|(10,[0,1,4,7],[737.0,1.0,4.0,3.0])|0          |
|(10,[0,1,7],[7.0,1.0,3.0])        |0          |
|(10,[0,1,7],[13.0,1.0,4.0])       |0          |
|(10,[0,1,9],[14.0,1.0,1.0])       |0          |
|(10,[0,1,9],[14.0,1.0,1.0])       |0          |
|(10,[1,7],[1.0,3.0])              |0          |
|(10,[0,1,7,9],[9.0,1.0,3.0,1.0])  |0          |
|(10,[0,1,9],[85.0,1.0,1.0])       |1          |
|(10,[0,1,7],[75.0,1.0,1.0])       |1          |
|(10,[0,1],[23.0,1.0])             |1          |
|(10,[0,1],[35.0,1.0])             |0          |
|(10,[0,1,9],[68.0,1.0,3.0])       |0          |
|(10,[0,1,4,9],[18.0,1.0,1.0,1.0]) |0          |
|(10,[0,1],[37.0,1.0])             |0          |
|(10,[0,1,9],[68.0,1.0,3.0])       |0          |
|(10,[0,1,6,7],[37.0,1.0,2.0,1.0]) |0          |
|(10,[0,1],[12.0,1.0

In [ ]:
df1.printSchema()

root
 |-- lead_time: integer (nullable = true)
 |-- hotel: string (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- booking_changes: integer (nullable = true)
 |-- deposit_type: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- required_car_parking_spaces: integer (nullable = true)
 |-- total_of_special_requests: integer (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- hotel_index: double (nullable = false)
 |-- deposit_type_index: double (nullable = false)
 |-- customer_type_index: double (nullable = false)
 |-- market_segment_index: double (nullable = false)



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
#Data Split (Train/Test Split)

In [ ]:
train_data,test_data=df_xy.randomSplit([0.8,0.2],seed=42)

In [ ]:
train_data.describe().show()

+-------+------------------+
|summary|       is_canceled|
+-------+------------------+
|  count|             95484|
|   mean|0.3708998366218424|
| stddev|0.4830482289893774|
|    min|                 0|
|    max|                 1|
+-------+------------------+



In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|        is_canceled|
+-------+-------------------+
|  count|              23906|
|   mean|0.36848489918848826|
| stddev|0.48240388968351644|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [ ]:
train_data.show()

+----------+-----------+
|  features|is_canceled|
+----------+-----------+
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
|(10,[],[])|          0|
+----------+-----------+
only showing top 20 rows



In [ ]:
test_data.show()

+--------------+-----------+
|      features|is_canceled|
+--------------+-----------+
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          0|
|    (10,[],[])|          1|
|    (10,[],[])|          1|
|    (10,[],[])|          1|
|    (10,[],[])|          1|
|(10,[0],[1.0])|          0|
|(10,[0],[1.0])|          1|
|(10,[0],[1.0])|          1|
+--------------+-----------+
only showing top 20 rows



In [ ]:
model=LinearRegression(featuresCol="features",labelCol="is_canceled")

In [ ]:
lr_model=model.fit(train_data)

Model Evalution


In [ ]:
prediction=lr_model.transform(test_data)

In [ ]:
prediction.select("prediction","is_canceled").show(20)

+------------------+-----------+
|        prediction|is_canceled|
+------------------+-----------+
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          0|
|0.4115108775753004|          1|
|0.4115108775753004|          1|
|0.4115108775753004|          1|
|0.4115108775753004|          1|
|0.4121365441198786|          0|
|0.4121365441198786|          1|
|0.4121365441198786|          1|
+------------------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="is_canceled", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)

In [ ]:
print(f"Root mean Squared Error (RMSE):{rmse}")

Root mean Squared Error (RMSE):0.39990611846766905


In [ ]:
r2_evaluate=RegressionEvaluator(labelCol="is_canceled",predictionCol="prediction",metricName="r2")
r2=r2_evaluate.evaluate(prediction)
print(f"R-squared (R2):{r2}")

R-squared (R2):0.31275330042042526


In [ ]:
from pyspark.sql.functions import count

In [ ]:
df1.groupBy("hotel").agg(count("*").alias("total_bookings")).show()

+------------+--------------+
|       hotel|total_bookings|
+------------+--------------+
|  City Hotel|         79330|
|Resort Hotel|         40060|
+------------+--------------+



In [ ]:
from pyspark.sql.functions import avg

In [ ]:
df1.groupBy("hotel").agg(avg("lead_time").alias("average_lead_time")).show()

+------------+------------------+
|       hotel| average_lead_time|
+------------+------------------+
|  City Hotel|109.73572419009201|
|Resort Hotel| 92.67568647029456|
+------------+------------------+



In [ ]:
from pyspark.sql.functions import sum

In [ ]:
df1.groupBy("hotel").agg(sum("is_canceled").alias("totel_cancellations")).show()

+------------+-------------------+
|       hotel|totel_cancellations|
+------------+-------------------+
|  City Hotel|              33102|
|Resort Hotel|              11122|
+------------+-------------------+



In [ ]:
df1.groupBy("market_segment").agg(count("*").alias("totel_mooking")).show()

+--------------+-------------+
|market_segment|totel_mooking|
+--------------+-------------+
| Offline TA/TO|        24219|
| Complementary|          743|
|      Aviation|          237|
|        Direct|        12606|
|     Undefined|            2|
|     Corporate|         5295|
|     Online TA|        56477|
|        Groups|        19811|
+--------------+-------------+



In [ ]:
# No code was provided to correct.
# Please provide the code you want me to correct.

**Data Preprocessing:**

*   **Data Cleaning:**  Handles missing values by dropping rows with `na.drop()`.  This is a simple approach; imputation might be more sophisticated in a real-world scenario.
*   **Feature Selection:** Selects relevant features for the model.
*   **Feature Engineering:**  Converts categorical features (hotel type, deposit type, customer type, market segment) into numerical representations using StringIndexer. This is crucial as linear regression requires numerical input.
*   **Vector Assembler:** Combines selected features into a single vector column named "features", which is the input for the linear regression model.

**Model Training and Evaluation:**

*   **Data Splitting:** Splits the data into 80% training and 20% testing sets.
*   **Linear Regression:** Trains a linear regression model using the training data.
*   **Model Evaluation:** Evaluates the model's performance on the test data using:
    *   **RMSE (Root Mean Squared Error):** Measures the average difference between the predicted and actual cancellation values. A lower RMSE indicates better accuracy. The specific RMSE value is printed in the output.
    *   **R-squared (R2):** Represents the proportion of variance in the target variable explained by the model.  A higher R2 suggests a better fit. The specific R2 value is printed in the output.

**Additional Analysis:**

*   **Aggregate Functions:**  The code performs aggregations on the data to provide insights into:
    *   Total bookings by hotel type.
    *   Average lead time by hotel type.
    *   Total cancellations by hotel type.
    *   Total bookings by market segment.

**Possible Improvements:**

*   **More Robust Data Cleaning:** Explore more advanced imputation techniques for missing values instead of simply removing rows.
*   **Feature Scaling:** Consider scaling numerical features (like "lead\_time") using StandardScaler or MinMaxScaler to improve model performance.
*   **Hyperparameter Tuning:**  Experiment with different hyperparameters for the linear regression model (e.g., regularization parameters) using techniques like cross-validation to optimize its performance.
*   **Model Selection:** Try different regression models (e.g., decision trees, random forests, gradient boosting) and compare their performance.
*   **Feature Importance:** Analyze the feature importance scores to understand which features are most influential in predicting cancellations.

**Overall Interpretation of the Results:**

The RMSE and R2 values provide a quantitative measure of the model's prediction accuracy.  The aggregate functions offer insights into the booking and cancellation patterns for different hotel types and market segments. However, without specific values for RMSE and R2 in the output, I cannot assess the quality of the model. Analyze the RMSE and R2 printed by the execution to understand the accuracy of the predictions.  A low RMSE and a high R2 value indicate a better-performing model.


SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-57-17437b0ca4d1>, line 1)